# Test Millera-Rabina

#### Autorzy:
- Mateusz Serek 
- Weronika Sadoch

### Załadujmy odpowiednie biblioteki

In [5]:
import random
import ipywidgets as widgets

## Wprowadzenie
Test pierwszości Millera-Rabina służy do sprawdzenia czy zadana liczba jest złożona, czy prawdopodobnie pierwsza.


## Podstawy matematyczne

#### Twierdzenie Eulera
Jeżeli: 
$$ NWD(x,n)=1 $$ 
To: 
$$ x^{\varphi(n)} \equiv 1 \text{ }(mod\text{ }n) $$

#### Małe twierdzenie Fermata

Liczba całkowita $p > 1 $ jest pierwsza wtedy i tylko wtedy gdy: 
 

$$ \forall a \not\equiv 0\text{ } (mod\text{ }p), \text{  } a^{p-1} \equiv 1 \text{ }(mod\text{ } p) $$


## Algorytm - wersja podstawowa
Dane są $n >= 5$
1. Szukamy takich liczb naturalnych $s, d$: $s \neq d$, gdzie $s$ jest nieparzyste oraz $n - 1  = 2 ^{s}$ * $ d$.
2. Wybierzmy liczbę losową $a$ taką, że $1 < a < n$
3. Niech $b = a ^{d}$. Jeżeli $b \equiv  1$ (mod $n$), zwróć prawdę.
4. W przeciwnym wypadku jeżeli $b^{2^{r}} \equiv -1$ (mod $n$), gdzie $\forall r$ $1 \le  r \le  s-1$, zwróć prawdę.
5. W przeciwnym wypadku zwróć fałsz

Zwrócona prawda oznacza, że liczba $n$ jest prawdopodobnie pierwsza, w przypadku fałszu liczba $n$ napewno nie jest liczbą pierwszą.

## Usprawnienie testu
Aby Zwiększyć prawdopodobieństwo trzeba zwiększyć liczbę prób wykonanych dla danej liczby.

## Implementacja Testu Millera-Rabina w SageMath

In [ ]:
def miller_rabbin(n: int, k: int) -> bool: 
    #n - testowana liczba
    #k - dokladnosc...

    def power_mod(a: int, b: int, c: int) -> int: #funkcja pomocnicza do obliczenia potęgi modulo
        result = 1
        for i in range(b):
            result *= a 
            result %= c 
        return result
    
    if n < 5:
        raise ValueError(f"n = {n} cannot be below 5")
        
    if k < 0:
        raise ValueError(f"k = {k} cannot be below 0")
        
    if n % 2 == 0:
        return False
    
    d = n - 1
    s = 0
    
    while d % 2 == 0: #Krok 1.
        s += 1
        d //= 2
        
    for p in range(k): # Po ulepszeniu algorytmu wykonywane jest wiele prób. 
        a = random.randint(2, n) #Krok 2.
        b = power_mod(a, d, n)
        
        for q in range(s): 
            y = (b * b) % n
            
            if y == 1 and b != 1 and b != n - 1: # -1 (mod n) = n - 1, warunek z kroku 4.
                return False
            
            b = y
            
        if y != 1: # W ulepszonej wersji algorytmu warunek z kroku 3. jest sprawdzany na końcu iteracji.
            return False
        
    return True  # Jeśli żadna próba nie wykluczy pierwszości, zwrócona zostanie prawda

### Sprawdźmy działanie algorytmu.

In [37]:
numbers = [random.randint(10 ** 5, 10 ** 6) for i in range(10)]
for i in numbers:
    current = miller_rabbin(i, 5)
    if current:
        print(f"Liczba {i} prawdopodobnie jest pierwsza")
    else:
        print(f"Liczba {i} napewno nie jest pierwsza")

Liczba 971063 prawdopodobnie jest pierwsza
Liczba 212741 napewno nie jest pierwsza
Liczba 902630 napewno nie jest pierwsza
Liczba 666049 napewno nie jest pierwsza
Liczba 766320 napewno nie jest pierwsza
Liczba 618806 napewno nie jest pierwsza
Liczba 598818 napewno nie jest pierwsza
Liczba 692651 prawdopodobnie jest pierwsza
Liczba 745100 napewno nie jest pierwsza
Liczba 940187 napewno nie jest pierwsza


## Złożoność obliczeniowa
lorem ipsum 

### Źródła
https://wstein.org/ent/ent.pdf <br>
https://en.wikipedia.org/wiki/Miller–Rabin_primality_test